In [29]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [30]:
us_zip = pd.read_csv('zip_code_database.csv')
ohio_zip = us_zip[us_zip['state'] == 'OH']
ohio_zip.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
18504,43001,STANDARD,0,Alexandria,NaN,NaN,OH,Licking County,America/New_York,740,NaN,US,40.08,-82.61,2440
18505,43002,PO BOX,0,Amlin,NaN,NaN,OH,Franklin County,America/New_York,614,NaN,US,40.07,-83.18,79
18506,43003,STANDARD,0,Ashley,NaN,NaN,OH,Delaware County,America/New_York,740,NaN,US,40.40,-82.95,2610
18507,43004,STANDARD,0,Blacklick,NaN,NaN,OH,Franklin County,America/New_York,614,NaN,US,40.02,-82.80,25320
18508,43005,PO BOX,0,Bladensburg,NaN,NaN,OH,Knox County,America/New_York,740,NaN,US,40.29,-82.28,274


In [31]:
ohio_zip = ohio_zip[ohio_zip['county'].isin(['Portage County', 'Geauga County', 'Summit County', 'Cuyahoga County'])]
ohio_zip.drop(['country', 'world_region', 'area_codes', 'timezone', 'state', 'decommissioned', 'type'], axis=1, inplace=True)
print(ohio_zip.shape)
ohio_zip.head()

(147, 8)


,zip,primary_city,acceptable_cities,unacceptable_cities,county,latitude,longitude,irs_estimated_population_2015
18998,44017,Berea,NaN,NaN,Cuyahoga County,41.36,-81.86,15450
18999,44021,Burton,NaN,NaN,Geauga County,41.47,-81.14,5890
19000,44022,Chagrin Falls,"Bentleyville, Moreland Hills, Moreland Hls","Chagrin Township, Hunting Valley, Munson Twp, ...",Cuyahoga County,41.43,-81.38,16620
19001,44023,Chagrin Falls,"Auburn Township, Auburn Twp",Bainbridge Township,Geauga County,41.38,-81.29,18230
19002,44024,Chardon,"Concord Township, Concord Twp",Kirtland,Geauga County,41.57,-81.20,22390


In [43]:
import folium
latitude = 41.5
longitude = -81.2
map_cle = folium.Map(location=[latitude, longitude], zoom_start=9)

for lat, lng, city in zip(ohio_zip['latitude'], ohio_zip['longitude'], ohio_zip['primary_city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cle)  
    
map_cle

In [46]:
CLIENT_ID = 'your-id' # your Foursquare ID
CLIENT_SECRET = 'your-secret' # your Foursquare Secret
ACCESS_TOKEN = 'your-access-token' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KKZN3HI5KKZC30UIROPFARM314L12UC2K01AGO3NXTKRT5PC
CLIENT_SECRET:GSBHCVYQ2AAI3M2ZF1MJ0IKDVBLK1IMLZU5RJHGAQ20JJ3MJ


In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
cle_venues = getNearbyVenues(names=ohio_zip['primary_city'],
                                   latitudes=ohio_zip['latitude'],
                                   longitudes=ohio_zip['longitude']
                                  )
cle_venues.head()

Berea
Burton
Chagrin Falls
Chagrin Falls
Chardon
Chesterland
East Claridon
Gates Mills
Huntsburg
Macedonia
Middlefield
Montville
Newbury
Northfield
North Olmsted
Novelty
Novelty
Parkman
Thompson
Twinsburg
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Lakewood
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Rocky River
Euclid
Cleveland
Cleveland
Cleveland
Cleveland
Beachwood
Euclid
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Independence
Euclid
North Royalton
Cleveland
Cleveland
Strongsville
Maple Heights
Olmsted Falls
Solon
Bay Village
Brecksville
Brookpark
Cleveland
Cleveland
Westlake
Bedford
Broadview Heights
Strongsville
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Cleveland
Atwater
Aurora
Barberton
Bath
Brady Lake
Clinton
Cuyahoga Falls
Cuyahoga Falls
Cuyahoga Falls
Stow
Garrettsville
Green
Hiram
Hudson
Hudson
Kent
Streetsboro


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berea,41.36,-81.86,Metroparks Wallace Lake,41.361085,-81.858418,Lake
1,Berea,41.36,-81.86,Quarry Rock Café,41.359784,-81.859522,Snack Place
2,Berea,41.36,-81.86,Baldwin Lake Viewing Area,41.360125,-81.857222,Scenic Lookout
3,Burton,41.47,-81.14,Quintealia's Tea Parlor,41.471695,-81.143931,Tea Room
4,Burton,41.47,-81.14,Century Village,41.469971,-81.143293,Historic Site


In [80]:
cle_coffee = cle_venues[cle_venues['Venue Category'] == 'Coffee Shop']
cle_coffee = cle_coffee.set_index('Neighbourhood')
cle_coffee = cle_coffee.drop_duplicates()
cle_coffee = cle_coffee.reset_index()
cle_coffee

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cleveland,41.51,-81.61,Starbucks,41.508160,-81.605880,Coffee Shop
1,Cleveland,41.51,-81.61,The Coffee House at University Circle,41.511554,-81.607338,Coffee Shop
2,Cleveland,41.48,-81.70,Duck Rabbit Coffee,41.483234,-81.700347,Coffee Shop
3,Cleveland,41.50,-81.56,Phoenix Coffee,41.497404,-81.564890,Coffee Shop
4,Cleveland,41.50,-81.56,Starbucks,41.500992,-81.565170,Coffee Shop
5,Cleveland,41.39,-81.74,Arabica Pearl Road,41.389361,-81.740208,Coffee Shop
6,Solon,41.38,-81.44,Starbucks,41.382004,-81.436530,Coffee Shop
7,Bay Village,41.48,-81.92,Mojo's Coffee and More,41.478504,-81.921396,Coffee Shop
8,Bay Village,41.48,-81.92,Java Bay,41.484278,-81.921694,Coffee Shop
9,Hiram,41.31,-81.14,Maggie's Doughnuts,41.309517,-81.145454,Coffee Shop


In [84]:
latitude = 41.5
longitude = -81.5
map_coffee = folium.Map(location=[latitude, longitude], zoom_start=9)

for lat, lng, city in zip(cle_coffee['Venue Latitude'], cle_coffee['Venue Longitude'], cle_coffee['Neighbourhood']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_coffee)  
    
map_coffee